In [1]:
import pandas as pd
from app import WinrateCollect2

In [6]:
wc = WinrateCollect2()
wc.update_stat()

In [2]:
# prepare heros stats
df = pd.read_pickle("position_stats_data")
df['r'] = 3+((df.g ** (2/3)) * (2*df.wr/100 - 1) / 10) 
#df['hero'] = df['hero'].str.replace('%20',' ')
#df = df.astype({'g': 'int32'})
df.head()

,hero,g,wr,pos,r
0,Abaddon,60,52.0,3,3.061305
1,Abaddon,362,49.3,5,2.928889
2,Alchemist,71,52.4,1,3.082303
3,Ancient Apparition,24,66.7,4,3.277899
4,Ancient Apparition,291,53.1,5,3.272262


In [4]:
# prepare heros matchups
df_m = pd.read_pickle("matchup_stats_data")
#df_m['hero'] = df_m['hero'].str.replace('%20',' ')
#df_m = df_m.astype({'w': 'int32'})
#df_m = df_m.astype({'l': 'int32'})
df_m.head()

,hero,matchup,w,l
0,Abaddon,Storm Spirit,16,4
1,Abaddon,Undying,11,2
2,Abaddon,Tinker,20,7
3,Abaddon,Tidehunter,6,0
4,Abaddon,Tusk,11,3


In [80]:
def get_matchup(enemy_heroes):
    matchup = None
    for e in enemy_heroes:
        tmp = df_m.loc[(df_m['hero'] == e)][['matchup', 'w', 'l']].set_index('matchup') 
        if matchup is not None:
            matchup = matchup.add(tmp, fill_value=0)
        else:
            matchup = tmp
    matchup = matchup[~matchup.index.isin(enemy_heroes)]
    center_g = 1
    center_lr = 1
    matchup = matchup.assign(g = matchup['w'] + matchup['l'])
    matchup = matchup.assign(lr = matchup['l'] / (matchup['w'] + matchup['l']))
    matchup = matchup.assign(g_qant = matchup['g'].rank(method='max', pct=True))
    matchup = matchup.assign(lr_qant = matchup['lr'].rank(method='max', pct=True))
    matchup = matchup.assign(qant_circle = (matchup['g_qant']-center_g)**2 + (matchup['lr_qant']-center_lr)**2)
    matchup = matchup.assign(rank = 1 - matchup['qant_circle'])
    df_top = matchup.sort_values(by=['qant_circle'],ascending=True)[['g', 'lr']]
    df_top = df_top.loc[df_top['lr'] > 0.49]
    return df_top

In [82]:
df_tmp = df.sort_values(['hero', 'g'], ascending=[True,False]).drop_duplicates(['hero'])
p1 = list(df_tmp.loc[(df['pos'] == 1)].hero)
p2 = list(df_tmp.loc[(df['pos'] == 2)].hero)
p3 = list(df_tmp.loc[(df['pos'] == 3)].hero)
p4 = list(df_tmp.loc[(df['pos'] == 4)].hero)
p5 = list(df_tmp.loc[(df['pos'] == 5)].hero)

# insert enemy heroes
enemies = ['Ancient Apparition', 'Hoodwink']
m = get_matchup(enemies)

# select pos
m[m.index.isin(p5)]

,g,lr
matchup,,
Abaddon,465.0,0.541935
Winter Wyvern,575.0,0.525217
Keeper of the Light,251.0,0.541833
Phoenix,208.0,0.533654
Treant Protector,158.0,0.544304
Disruptor,196.0,0.515306
Enchantress,165.0,0.496970
Bane,99.0,0.555556
Io,108.0,0.527778


In [5]:
pos = 2
# & (df['wr'] >= 52)
min_g = df.loc[(df['pos'] == pos)]['g'].max() / 10
df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g) ]
df_top3 = df1.sort_values(by=['r'],ascending=False)
df_top3[:20]

,hero,g,wr,pos,r
192,Queen of Pain,2429,53.8,2,4.373300
28,Broodmother,301,60.9,2,3.979116
238,Templar Assassin,534,55.3,2,3.697697
74,Ember Spirit,1238,52.5,2,3.576479
115,Leshrac,746,52.2,2,3.361919
11,Batrider,904,51.7,2,3.317876
246,Tinker,1098,51.0,2,3.212862
183,Puck,2410,50.5,2,3.179754
232,Storm Spirit,1240,50.3,2,3.069252
171,Outworld Devourer,603,49.0,2,2.857250


In [24]:
p1 = list(df_tmp.loc[(df['pos'] == 1)].hero)

In [6]:
pos = 1
# & (df['wr'] >= 52)
min_g = df.loc[(df['pos'] == pos)]['g'].max() / 100
df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g) ]
df_top3 = df1.sort_values(by=['wr'],ascending=False)
df_top3

,hero,g,wr,pos
201,Shadow Fiend,45,78.6,1
186,Razor,130,59.6,1
18,Bounty Hunter,33,58.3,1
210,Slark,495,58.0,1
121,Luna,2563,55.1,1
24,Bristleback,104,54.8,1
87,Gyrocopter,1500,54.7,1
133,Medusa,1084,54.6,1
80,Faceless Void,1164,54.1,1
115,Lifestealer,388,53.3,1


In [8]:
'''
# old version
def tierS(pos):
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 3
    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g) & (df['wr'] >= 52)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3

def tierA(pos):
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 4
    df2 = tierS(pos)
    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g)  & (df['wr'] >= 49)]
    df1 = df1.sort_values(by=['wr'],ascending=False).head(14)
    df_top8 = pd.concat([df1, df2]).drop_duplicates(keep=False)
    return df_top8.head(11)

def topWR():
    min_g = df['g'].max() / 40
    df_top3 = df.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)

def topTier():
    min_g = df['g'].max() / 10
    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3.head(15)
'''

"\n# old version\ndef tierS(pos):\n    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 3\n    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g) & (df['wr'] >= 52)]\n    df_top3 = df1.sort_values(by=['wr'],ascending=False)\n    return df_top3\n\ndef tierA(pos):\n    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 4\n    df2 = tierS(pos)\n    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g)  & (df['wr'] >= 49)]\n    df1 = df1.sort_values(by=['wr'],ascending=False).head(14)\n    df_top8 = pd.concat([df1, df2]).drop_duplicates(keep=False)\n    return df_top8.head(11)\n\ndef topWR():\n    min_g = df['g'].max() / 40\n    df_top3 = df.sort_values(by=['wr'],ascending=False)\n    return df_top3.head(10)\n\ndef topTier():\n    min_g = df['g'].max() / 10\n    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]\n    df_top3 = df1.sort_values(by=['wr'],ascending=False)\n    return df_top3.head(15)\n"

In [5]:
# make games count less valuable
#center_g = 0.9

# make win rate less valuable
#center_wr = 0.9

center_g = 1
center_wr = 1


#.style.hide_index()
def tierS(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[df1['qant_circle'] <= 0.0625]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top

def tierA(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[(df1['qant_circle'] <= 0.25) & (df1['qant_circle'] > 0.0625)]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top

def tierB_plus(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[(df1['qant_circle'] > 0.25)]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top

def topWR_lowpick():
    df_top3 = df.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


def topHiddenImba_pos(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g/2)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[df1['qant_circle'] <= 0.25]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top.head(3)
    
def topWinrateOverallTiers():
    min_g = df['g'].max() / 10
    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


In [6]:
topWinrateOverallTiers()

,hero,g,wr,pos
9,Batrider,192,66.3,2
59,Earthshaker,205,62.8,4
124,Monkey King,80,62.2,2
108,Lycan,149,61.1,3
204,Sven,261,60.3,1
129,Naga Siren,67,60.0,1
149,Pangolier,113,59.8,4
219,Tiny,59,58.2,4
13,Beastmaster,423,57.6,3
29,Chen,69,57.4,4


In [7]:
topWR_lowpick()

,hero,g,wr,pos
232,Venomancer,7,100.0,2
82,Io,13,83.3,4
186,Silencer,11,80.0,2
119,Mirana,10,77.8,2
101,Lina,19,76.5,5
190,Skywrath Mage,12,72.7,2
220,Treant Protector,20,72.2,4
164,Pugna,23,71.4,4
46,Dazzle,23,70.0,4
241,Visage,14,69.2,4


In [8]:
tierS(1)

,hero,g,wr,rank
105,Luna,369,54.1,0.965517
204,Sven,261,60.3,0.957194


In [9]:
tierA(1)

,hero,g,wr,rank
194,Slark,565,53.8,0.923900
68,Faceless Void,209,56.5,0.919144
229,Ursa,152,54.8,0.780024
127,Morphling,173,51.6,0.759810


In [10]:
tierB_plus(1)

,hero,g,wr,rank
4,Anti-Mage,265,49.2,0.747919
123,Monkey King,262,49.2,0.737218
152,Phantom Lancer,350,49.1,0.684899
106,Lycan,113,55.6,0.645660
199,Spectre,152,49.6,0.598098
86,Juggernaut,123,51.8,0.576694
117,Medusa,233,47.9,0.556480
74,Gyrocopter,425,45.4,0.523187
207,Templar Assassin,71,54.0,0.481570
247,Weaver,107,50.5,0.443520


In [11]:
tierS(2)

,hero,g,wr,rank
9,Batrider,192,66.3,0.940833


In [12]:
tierA(2)

,hero,g,wr,rank
165,Queen of Pain,273,54.3,0.914536
243,Void Spirit,371,51.4,0.873630
64,Ember Spirit,401,51.0,0.834916
94,Leshrac,172,51.3,0.783784
124,Monkey King,80,62.2,0.751644


In [13]:
tierS(3)

,hero,g,wr,rank
13,Beastmaster,423,57.6,0.9856
108,Lycan,149,61.1,0.9640
135,Night Stalker,227,56.1,0.9500
43,Dawnbreaker,252,55.4,0.9388


In [14]:
tierA(3)

,hero,g,wr,rank
93,Legion Commander,160,54.1,0.8844
233,Venomancer,103,57.0,0.8844
148,Pangolier,245,53.4,0.8780
38,Dark Seer,219,52.3,0.8560
53,Dragon Knight,125,53.8,0.8300
111,Magnus,88,55.1,0.8028


In [15]:
tierS(4)

,hero,g,wr,rank
59,Earthshaker,205,62.8,0.966426


In [16]:
tierA(4)

,hero,g,wr,rank
56,Earth Spirit,217,55.6,0.894112
121,Mirana,346,53.4,0.867252
149,Pangolier,113,59.8,0.825413
77,Hoodwink,234,52.4,0.824380
201,Spirit Breaker,132,55.8,0.793388
175,Rubick,363,51.2,0.793388
136,Nyx Assassin,185,53.3,0.788223


In [17]:
topHiddenImba_pos(4)

,hero,g,wr,rank
153,Phoenix,42,63.2,0.970041
149,Pangolier,113,59.8,0.961777
254,Winter Wyvern,52,59.6,0.954029


In [18]:
tierS(5)

,hero,g,wr,rank
90,Keeper of the Light,188,57.1,0.992604
2,Abaddon,254,56.2,0.986317
49,Disruptor,179,55.3,0.958210
8,Bane,327,53.8,0.955251
30,Chen,187,53.6,0.937500


In [19]:
tierA(5)

,hero,g,wr,rank
37,Crystal Maiden,132,55.3,0.931583
71,Faceless Void,111,56.9,0.896080
73,Grimstroke,157,52.7,0.886834
142,Oracle,111,52.0,0.798447
18,Bounty Hunter,105,52.6,0.785503
85,Jakiro,124,51.3,0.779586


In [20]:
topHiddenImba_pos(5)

,hero,g,wr,rank
47,Dazzle,67,54.2,0.962648
71,Faceless Void,111,56.9,0.953772
196,Snapfire,48,55.8,0.951923


In [25]:
import os
from datetime import datetime
fold = 'history/'+ datetime.today().strftime('%Y_%m_%d__%Hh_%Mm')
os.mkdir(fold)

In [26]:
import dataframe_image as dfi

In [27]:
tierS(1).dfi.export(fold + '/1_TierS.png')
tierA(1).dfi.export(fold + '/1_TierA.png')
tierB_plus(1).dfi.export(fold + '/1_TierBplus.png')
topWinrateOverallTiers().dfi.export(fold + '/topWinrate.png')
topWR_lowpick().dfi.export(fold + '/topLowpick.png')
tierS(2).dfi.export(fold + '/2_TierS.png')
tierA(2).dfi.export(fold + '/2_TierA.png')
tierS(3).dfi.export(fold + '/3_TierS.png')
tierA(3).dfi.export(fold + '/3_TierA.png')
tierS(4).dfi.export(fold + '/4_TierS.png')
tierA(4).dfi.export(fold + '/4_TierA.png')
topHiddenImba_pos(4).dfi.export(fold + '/4_HiddenImba.png')
tierS(5).dfi.export(fold + '/5_TierS.png')
tierA(5).dfi.export(fold + '/5_TierA.png')
topHiddenImba_pos(5).dfi.export(fold + '/5_HiddenImba.png')

In [55]:
from IPython.display import display, HTML

css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

In [61]:
display(tierS(1))
display(tierA(1))
#display(tierB_plus(1))

,hero,g,wr,rank
121,Luna,2563,55.1,0.994898
87,Gyrocopter,1500,54.7,0.983418
80,Faceless Void,1164,54.1,0.947704
133,Medusa,1084,54.6,0.947704


,hero,g,wr,rank
171,Phantom Lancer,1446,50.8,0.834184
244,Ursa,831,50.6,0.753827


In [7]:
df1 = df.loc[(df['pos'] == 1)]
df1 = df1.sort_values(by=['g'],ascending=False)
df1

,hero,g,wr,pos
117,Luna,2542,55.6,1
213,Terrorblade,1840,48.2,1
164,Phantom%20Lancer,1422,51.3,1
84,Gyrocopter,1238,55.3,1
77,Faceless%20Void,1114,52.7,1
204,Spectre,956,45.7,1
130,Medusa,952,53.1,1
232,Ursa,856,50.0,1
137,Morphling,852,45.9,1
264,Wraith%20King,704,48.8,1


In [32]:
df1.describe()

,g,wr,g_qant,wr_qant
count,40.000000,40.000000,40.000000,40.000000
mean,437.750000,48.987500,0.513125,0.514375
std,408.129991,6.155306,0.291492,0.291486
min,104.000000,34.700000,0.025000,0.025000
25%,164.750000,44.275000,0.268750,0.268750
50%,325.000000,49.550000,0.512500,0.512500
75%,479.000000,51.675000,0.756250,0.756250
max,1964.000000,66.700000,1.000000,1.000000


In [52]:
# make games count less valuable
#center_g = 0.9

# make win rate less valuable
#center_wr = 0.9

center_g = 1
center_wr = 1


#.style.hide_index()
def tiers(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df1['tier'] = ''
    df1['tier'] = df1['qant_circle'].apply(lambda x: 'S' if x <= 0.0625 else 
                                           ('A' if (x <= 0.25) & (x > 0.0625) else 
                                           ('B' if (x <= 0.5625) & (x > 0.25) else 'C+')))
    
    df_top = df1.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank', 'tier']]
    df_top.index = range(1, len(df1.index) + 1)
    return df_top


def topWR_lowpick():
    df_top3 = df.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


def topHiddenImba_pos(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g/2)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[df1['qant_circle'] <= 0.25]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    df_top.index = range(1, len(df2.index) + 1)
    return df_top.head(10)
    
def topWinrateOverallTiers():
    min_g = df['g'].max() / 10
    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


In [53]:
from IPython.display import display, HTML

css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

In [54]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
subset = ['rank']

a = tiers(5)
a = a.style.background_gradient('RdYlGn', subset = subset)
a

,hero,g,wr,rank,tier
1,Winter Wyvern,1641,55.200000,0.992695,S
2,Phoenix,517,53.900000,0.934989,A
3,Hoodwink,767,53.100000,0.934989,A
4,Abaddon,1271,52.300000,0.924032,A
5,Ancient Apparition,1986,51.500000,0.894814,A
6,Nyx Assassin,429,53.000000,0.852447,A
7,Ogre Magi,377,55.600000,0.810080,A
8,Treant Protector,501,50.500000,0.776479,A
9,Lion,820,49.900000,0.751644,A
10,Io,332,63.200000,0.736304,B


In [40]:
display(tiers(4)), 
display(topHiddenImba_pos(4))

,hero,g,wr,rank,tier
1,Nyx Assassin,1354,58.4,0.992188,S
2,Dark Willow,963,56.8,0.966797,S
3,Mirana,1819,53.2,0.952148,S
4,Lion,1262,52.9,0.928711,A
5,Pudge,630,51.5,0.823242,A
6,Winter Wyvern,380,64.8,0.717773,B
7,Skywrath Mage,545,50.9,0.716797,B
8,Windranger,717,50.1,0.702148,B
9,Shadow Shaman,359,54.3,0.659180,B
10,Earth Spirit,869,47.7,0.612305,B


,hero,g,wr,rank
1,Winter Wyvern,380,64.8,0.999023
2,Shadow Shaman,359,54.3,0.971680
3,Dawnbreaker,299,54.2,0.902344
4,Dark Willow,963,56.8,0.873047
5,Pudge,630,51.5,0.854492
6,Riki,175,55.1,0.843750
7,Phoenix,329,51.3,0.811523
8,Skywrath Mage,545,50.9,0.810547
9,Bounty Hunter,386,50.7,0.808594
10,Invoker,98,63.4,0.807617
